In [1]:
using DataFrames
using Chemfiles
@everywhere using DistributedArrays

home = "/home/german/labo/18/egfr/model/"
rehome = "/home/german/labo/18/egfr/modos/"
aa = 277

INFO: Recompiling stale cache file /home/german/.julia/lib/v0.6/DistributedArrays.ji for module DistributedArrays.


277

In [2]:
function tognm(vtor_anm::Array{Float64, 1})
    const n_vec_anm = length(vtor_anm)
    nvec_gnm = Int64
    try
        nvec_gnm = convert(Int64, n_vec_anm / 3);
    catch e
        error("Input vector length not divisible by 3.  ", e)
    end
    
    vtor_gnm = Array{Float64}(n_vec_gnm)
    vtor_anm =  vtor_anm.^2
    k = 0
    for i=1:3:n_vec_anm
        k += 1
        vtor_gnm[k] = sqrt(vtor_anm[i] + vtor_anm[i+1] + vtor_anm[i+2])
    end
    
    return vtor_gnm
end

tognm (generic function with 1 method)

In [3]:
function res_3_to_1(in_3::AbstractString)
    
    res = lowercase(in_3)
    out_1 = AbstractString
    
    if res == "ala"
        out_1 = "A"
    elseif res == "arg"
        out_1 = "R"
    elseif res == "asn"
        out_1 = "N"
    elseif res == "asp"
        out_1 = "D"
    elseif res == "cys"
        out_1 = "C"
    elseif res == "gln"
        out_1 = "Q"
    elseif res == "glu"
        out_1 = "E"
    elseif res == "gly"
        out_1 = "G"
    elseif res == "his"
        out_1 = "H"
    elseif res == "ile"
        out_1 = "I"
    elseif res == "leu"
        out_1 = "L"
    elseif res == "lys"
        out_1 = "K"
    elseif res == "met"
        out_1 = "M"
    elseif res == "phe"
        out_1 = "F"
    elseif res == "pro"
        out_1 = "P"
    elseif res == "ser"
        out_1 = "S"
    elseif res == "thr"
        out_1 = "T"
    elseif res == "trp"
        out_1 = "W"
    elseif res == "tyr"
        out_1 = "Y"
    elseif res == "val"
        out_1 = "V"
    else
        error("Unknown aminoacid: ", in_3)
    end
    
    return out_1
end

function res_3_to_1(in_vec::Array{String, 1})

    out_vec = Array{String, 1}(length(in_vec))
    i = 0
    for each in in_vec
        i+=1
        try 
            out_vec[i] = res_3_to_1(each)
        catch e
            error("Malformed array list.\nCaught error: $e")
        end
    end
    
    return out_vec
end

res_3_to_1 (generic function with 2 methods)

In [4]:
function seq_from_top(in_top::Chemfiles.Topology)
    # Get 1 letter code amino acid list from a topology
    aa = convert(Int64, count_residues(in_top))
    ids = Array{Int64, 1}(aa)

    for i=1:aa
        ids[i] = id(Residue(in_top, i-1))
    end
    ids_sorted_indices = sortperm(ids);

    seq_3 = Array{String, 1}(aa)
    i = 0
    for each in ids_sorted_indices
        i+=1
        seq_3[i] = name((Residue(in_top, each-1)))
    end
    seq_1 = res_3_to_1(seq_3);
    
    return seq_1
end

seq_from_top (generic function with 1 method)

# Correlaciono Bfactors experimentales y teóricos de residuos no-missing

In [5]:
tmp = readdlm(string(home, "/pdbs.list"))
pdbs_list = convert(Array{String, 1}, reshape(tmp, length(tmp)));

## Leo alineamiento

In [6]:
alineamientos = Array{Array{Any, 2}, 1}(length(pdbs_list))
indices_mut = Array{Array{Int64, 1}, 1}(length(pdbs_list))
indices_mis = Array{Array{Int64, 1}, 1}(length(pdbs_list))

j = 0

for each in pdbs_list
    j += 1
    println(each) 
    
    in_seq_canonical = Array{AbstractString, 1}
    in_seq_each = Array{AbstractString, 1}
    in_mut_each = Array{AbstractString, 1}

    seq_canonical = Array{AbstractString, 1}
    seq_each = Array{AbstractString, 1}
    mut_each = Array{AbstractString, 1}
    
    
    # Leo archivo.
    in_filename = string(home, each, "/alignment_", each)
    alignment_file = open(in_filename)
    lineas = readlines(alignment_file)

    k = 0
    # Ahora leo todas las líneas y tomo los arrays en bruto
    for linea in lineas
        k+=1
        inicio = 31
        final = length(lineas) - 3
    
        if inicio < k < final
            n_linea = k - 31
            altura = ((n_linea - 1 + 4) % 4)
        
            if altura == 0
                in_seq_canonical = [ in_seq_canonical ; linea[22:end-7] ]
            elseif altura == 1
                in_mut_each = [ in_mut_each ; linea[22:end] ]
            elseif altura == 2
                in_seq_each = [ in_seq_each ; linea[22:end-7] ]
            end
        end
    end
    
    # Paso las secuencias en bruto a arrays bien formados.
    for i = 2:length(in_seq_canonical)
        tmp_canonical = [ in_seq_canonical[i][j] for j = 1:length(in_seq_canonical[i]) ]
        seq_canonical = [ seq_canonical ; tmp_canonical]
    
        tmp_each = [ in_seq_each[i][j] for j = 1:length(in_seq_each[i]) ]
        seq_each = [ seq_each ; tmp_each]

        tmp = [ in_mut_each[i][j] for j = 1:length(in_mut_each[i]) ]
        mut_each = [ mut_each ; tmp]
    end

    # Determino q pdbs tienen mutaciones y en q posiciones.
    tmp_indices_mut = findin(mut_each, ".")
    indices_mut[j] = tmp_indices_mut
    
    # Determino las regiones a modelar
    tmp_indices_mis = findin(mut_each, " ") .- 1
    indices_mis[j] = tmp_indices_mis

    # Guardo todo
    alineamiento = [ seq_canonical seq_each mut_each ]
    alineamientos[j] = alineamiento 
end

1M14_A
1M17_A
1XKK_A
2EB2_A
2GS2_A
2GS6_A
2GS7_A
2ITN_A
2ITP_A
2ITU_A
2ITX_A
2ITZ_A
2J5F_A
2RGP_A
3BEL_A
3GOP_A
3GT8_A
3GT8_C
3IKA_A
3IKA_B
3LZB_A
3UG1_A
3VJN_A
3W2O_A
3W2R_A
3W2S_A
3W32_A
3W33_A
4G5J_A
4I1Z_A
4I22_A
4I23_A
4I24_A
4LI5_A
4LQM_A
4R3P_A
4R5S_A
4RJ4_A
4ZAU_A
4ZJV_A
5C8K_A
5CAO_A
5CAP_A
5CAV_A
5CNN_A
5CNN_B
5CNO_A
5CZH_A
5EDP_A
5EM5_A
5EM6_A
5EM7_A
5EM8_A
5HG5_A
5HG7_A
5HG8_A
5HIB_A
5HIC_A


### Renumero los indices missings. La numeración original es según
### las posiciones del alineamiento. Quiero obtener la numeración según
### la sequencia canonical

In [7]:
mod_indices_mis = Array{Array{Int64, 1}, 1}(length(pdbs_list))

for i = 1:length(pdbs_list)
    tmp_indices = Array{Int64}[]
    
    # La secuencia a modelar empieza y termina con leucina
    pre_mask = findin(alineamientos[i][:, 1], "L")
    mask = pre_mask[1] .< indices_mis[i] .< pre_mask[end]
    mod_indices_mis[i] = indices_mis[i][mask] .- pre_mask[1] .+ 2
    # Ya tengo los índices de missings numerados según la seq canonical.
end

## Determino la posición en la q empieza y termina la secuencia de relevancia

In [8]:
tmp_seq = zeros(317, length(pdbs_list))

starts = Array{Int64}(length(pdbs_list))
ends = Array{Int64}(length(pdbs_list))
check_mask_start = ['L' ; 'L' ; 'R' ]
check_mask_end = ['R' ; 'Y' ; 'L' ]

for j = 1:length(pdbs_list)
    # Saco 1 pq los arrays empiezan con un elemento undef
    tmp_struct = alineamientos[j][2:end, 2]
    tmp_struct = tmp_struct[tmp_struct .!= '-']
    for i = 1:50
        if check_mask_start == tmp_struct[i:i+2]
            starts[j] = i
            break
        end
    end
    for i = 250:length(tmp_seq)
        if check_mask_end == tmp_struct[i:i+2]
            ends[j] = i + 2
            break
        end
    end
end

## Leo bfactors de PDBs originales, sin completar

In [9]:
# 3W32_A, con 317 aa's, es el modelo más largo
proto_bfactors = zeros(317, length(pdbs_list))
i = 3
for i = 1:length(pdbs_list)
    each = pdbs_list[i]
    in_filename = string(home, each, "/", each, ".pdb")
    in_file = open(in_filename)

    j = 0
    for linea in eachline(in_file)
        try
            if linea[14:15] == "CA"
                j += 1
                proto_bfactors[j, i] = parse(Float64, linea[61:66])
            end
        catch e
            break
        end
    end
    
end

## Adapto los Bfactors p/ q correspondan con la secuencia de relevancia

In [10]:
bfactors = Array{Array{Float64, 1}}(length(pdbs_list))

for i = 1:length(pdbs_list)    
    tmp_mask = proto_bfactors[:, i] .!= 0
    bfactors[i] = proto_bfactors[tmp_mask, i][starts[i]:ends[i]];
end

## Obtengo listado de aminoácidos nonmissing p/ c/ struct

In [11]:
nonmissing = Array{Array{Int64, 1}}(length(pdbs_list))
resto = 0

for i = 1:length(pdbs_list)
    n_missings = length(mod_indices_mis[i])
    nonmissing[i] = Array{Int64, 1}(aa - n_missings)
    ii = 1
    
    if n_missings == 0
        nonmissing[i] = collect(1:aa)
        continue
    end
    
    for j = 1:mod_indices_mis[i][1]-1
        nonmissing[i][ii] = j
        ii += 1
    end
    for k = 1:length(mod_indices_mis[i])
        try
            for j = mod_indices_mis[i][k]+1:mod_indices_mis[i][k+1]-1
                nonmissing[i][ii] = j
                ii += 1
            end
        catch
            for j = mod_indices_mis[i][k]+1:aa
                nonmissing[i][ii] = j
                ii += 1
            end
        end
    end

    if findin(nonmissing[i], mod_indices_mis[i]) != []
        println(i, " mal! ", pdbs_list[i])
    end
end

## Leo los Bfactors teóricos

In [12]:
proto_teo_bfactors = Array{Array{Float64, 1}}(length(pdbs_list))
for i = 1:length(pdbs_list)
    each = pdbs_list[i]
    in_filename = string(rehome, each, "/bfteo_", each)
    proto_teo_bfactors[i] = readdlm(in_filename)[:, 2]
end

## Ahora si, obtengo correlaciones entre Bfactors teóricos y experimentales

In [18]:
corr_bftors = Array{Float64}(length(pdbs_list))
for i = 1:length(pdbs_list)
    corr_bftors[i] = cor(bfactors[i], proto_teo_bfactors[i][nonmissing[i]])
end

## Filtro las structs q no dan buena correlación

In [13]:
# Ejecutar este box 1 sola vez!
malos_pdbs = findin(corr_bftors .< 0.4, 1)
tmp_mask = repeat([true], inner = length(pdbs_list))
tmp_mask[malos_pdbs] = false

pdbs_list = pdbs_list[tmp_mask]
mod_indices_mis = mod_indices_mis[tmp_mask]
bfactors = bfactors[tmp_mask];

# Bfactors de residuos del pocket presentes en todas las structs


In [14]:
tmp = readdlm(string(rehome, "disp_aa_pocket"))
disp_aa_pocket = convert(Array{Int64, 1}, reshape(tmp, length(tmp)))
aa_pocket = disp_aa_pocket .- 702;

## Determino aminoácidos de pocket q no están missing en ninguna struct

In [15]:
indices_aa_pocket_missing = Array{Int64,1}(0)
for i = 1: length(pdbs_list)
    indices_aa_pocket_missing = [ indices_aa_pocket_missing ; findin(mod_indices_mis[i], aa_pocket) ]
end
indices_aa_pocket_missing = sort(unique(indices_aa_pocket_missing));

## Obtengo la lista de residuos de pocket no missing

In [16]:
tmp_mask = repeat([true], inner = length(aa_pocket))
tmp_mask[indices_aa_pocket_missing] = false

nonmis_aa_pocket = aa_pocket[tmp_mask]
nonmis_disp_aa_pocket = disp_aa_pocket[tmp_mask];

## Filtro y me quedo con los Bfactors de residuos de pocket entero y los no missing

In [17]:
bfactors_nonmis_pocket = Array{Array{Float64}, 1}(length(pdbs_list))
bfactors_pocket = Array{Array{Float64}, 1}(length(pdbs_list))

for i = 1:length(pdbs_list)
    bfactors_nonmis_pocket[i] = Array{Float64}(0)
    bfactors_nonmis_pocket[i] = bfactors[i][nonmis_aa_pocket]
    
    bfactors_pocket[i] = Array{Float64}(0)
    bfactors_pocket[i] = bfactors[i][aa_pocket]
end

## Guardo estos Bfactors p/ ponderar los grammianos

In [18]:
new_bfactors_nonmis_pocket = Array{Float64, 2}(length(nonmis_aa_pocket), length(pdbs_list))
new_bfactors_pocket = Array{Float64, 2}(length(aa_pocket), length(pdbs_list))


# Paso array de arrays a Array 2D
[ new_bfactors_nonmis_pocket[:, i] = bfactors_nonmis_pocket[i] for i = 1:length(pdbs_list) ];
[ new_bfactors_pocket[:, i] = bfactors_pocket[i] for i = 1:length(pdbs_list) ];

# Paso Array 2D a DataFrame con nombres
df_bfactors_nonmis_pocket = names!(DataFrame(new_bfactors_nonmis_pocket), 
    [ Symbol(pdbs_list[i]) for i = 1:length(pdbs_list) ]);
df_new_bfactors_pocket = names!(DataFrame(new_bfactors_pocket), 
    [ Symbol(pdbs_list[i]) for i = 1:length(pdbs_list) ]);

# Guardo los Bfactors p/ ponderación
writetable(string(rehome, "pesos_nonmis"), df_bfactors_nonmis_pocket, separator = '\t')
writetable(string(rehome, "pesos"), df_new_bfactors_pocket, separator = '\t')

In [19]:
# Guardo la lista de los residuos de pocket entero y nonmissing
writedlm(string(rehome, "aa_pocket"), aa_pocket)
writedlm(string(rehome, "nonmis_aa_pocket"), nonmis_aa_pocket)

writedlm(string(rehome, "nonmis_aa_disp_pocket"), nonmis_disp_aa_pocket)

## Guardo Bfactors experimentales e indices de nonmissing p/ el cálculo de amplitudes de modos

In [22]:
for i = 1:length(pdbs_list)
    each = pdbs_list[i]
    writedlm(string(rehome, each, "/", "exp_bf_", each), bfactors[i])
    writedlm(string(rehome, each, "/", "nonmissing_", each), nonmissing[i])
end

# Checkeo 4I22_A y 4I24_A

In [96]:
tmp = readdlm(string(rehome, "4I22_A_freq"))[8:end, 1]
frq_i22 = convert(Array{Float64, 1}, tmp)

frq_i24 = readdlm(string(rehome, "4I24_A_freq"))[8:end, 1]
frq_i24 = convert(Array{Float64, 1}, tmp)

mod_i22 = readdlm(string(rehome, "4I22_A_mods"))
mod_i24 = readdlm(string(rehome, "4I24_A_mods"));

In [110]:
tempo_0 = mapslices(x -> tognm(x), mod_i22, 1) * diagm(1 ./ frq_i22)
pseudo_teo_bf_i22 = mapslices(x -> sum(x), tempo_0, 2)[:, 1]
cor(bfactors[i], pseudo_teo_bf_i22[nonmissing[i]])

0.597731227442094

In [111]:
tempo_0 = mapslices(x -> tognm(x), mod_i24, 1) * diagm(1 ./ frq_i24)
pseudo_teo_bf_i24 = mapslices(x -> sum(x), tempo_0, 2)[:, 1]
cor(bfactors[i], pseudo_teo_bf_i24[nonmissing[i]])

0.5369498468937741